Вариационная модель звезды

Как параметр задана масса звезды M.
Звезда разбита на N сферических слоев.
Также задано распределение водорода по слоям.
Имеется N+1 точка - расстояния от центра до i-го слоя.
Неизвестными являются массивы расстояний r[i] и температур t[i]

Date of creation: 2.07.2020

Last updated: 3.07.2020

In [1]:
import math
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import physics as ph

Physics version 1.2


In [3]:
from Sun import SSM18 as s

SSM18 version 1.1


In [15]:
pi = math.pi
dimM = 18 # N+1 - число точек модели (сферических слоев + 1)
Z = s.Z

In [16]:
df1 = s.load_data()
df1

,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure
0,0.0000,0.000,15500000.0,1.563000e+02,0.000,0.355,1.1,0.0,2.381543e+17
1,0.0099,0.046,14800000.0,1.339000e+02,0.079,0.417,1.2,0.0,2.075100e+17
2,0.0385,0.076,13800000.0,1.081000e+02,0.264,0.497,1.3,0.0,1.685436e+17
3,0.1038,0.113,12400000.0,7.890000e+01,0.555,0.592,1.4,0.0,1.201441e+17
4,0.1620,0.138,11400000.0,6.320000e+01,0.718,0.641,1.6,0.0,9.212529e+16
5,0.2100,0.156,10800000.0,5.360000e+01,0.809,0.668,1.7,0.0,7.563500e+16
6,0.2580,0.173,10200000.0,4.570000e+01,0.874,0.688,1.8,0.0,6.186836e+16
7,0.3100,0.190,9600000.0,3.850000e+01,0.921,0.702,1.9,0.0,4.958997e+16
8,0.3900,0.217,8770000.0,2.940000e+01,0.964,0.716,2.1,0.0,3.496777e+16
9,0.4700,0.245,8000000.0,2.210000e+01,0.986,0.724,2.4,0.0,2.412367e+16


In [17]:
x = df1['Hydrogen'].values
r = df1['Radius'].values*s.R
m = df1['Mass'].values*s.M
t = df1['Temperature'].values
# The rest may be computed from the above variables.
d1 = df1['Density'].values
l1 = df1['Luminosity'].values*s.L
p1 = df1['Pressure'].values
k1 = df1['Opacity'].values

In [18]:
d = np.zeros(dimM)
l = np.zeros(dimM)
p = np.zeros(dimM)
k = np.zeros(dimM)

In [19]:

d[0] = 3*m[1]/4/pi/pow(r[1], 3)
d[1] = 0.5*d[0] + 0.5*(m[2] - m[1])/4/pi/pow((r[1] + r[2])/2, 2)/(r[2] - r[1])
for i in range(2, dimM-1):
    d[i] = ((m[i+1] - m[i])/pow(r[i], 2)/(r[i+1] - r[i]) + (m[i] - m[i-1])/pow(r[i], 2)/(r[i] - r[i-1]))/8/pi
d[dimM-1] = 0.
for i in range(dimM):
    p[i] = ph.Pressure(d[i], t[i], x[i], 1 - Z - x[i], Z)
    k[i] = ph.opacity(d[i], t[i], x[i], 1 - Z - x[i], Z)
l[0] = 0.
for i in range(1, dimM):
    l[i] = l[i-1] + (m[i] - m[i-1])*ph.Etot(d[i], t[i], x[i], 1 - Z - x[i], Z)/0.5879669006862714

In [20]:
for i in range(dimM):
    print(d1[i], d[i])

156.3 143.31724392258448
133.9 131.82731891579235
108.1 110.5196328708319
78.9 75.27591039108262
63.2 61.593351300072484
53.6 52.9814886114439
45.7 46.15772233028045
38.5 39.174516872533815
29.4 29.026649658852026
22.1 21.611853934197924
16.1 15.40830508890182
8.03 7.872433275180026
2.85 2.8791093599090924
0.773 0.8921474141681306
0.338 0.3571566041760261
0.169 0.16679284458681545
0.05 0.05220671087255304
2.8e-07 0.0


In [21]:
for i in range(dimM):
    print(d1[i], d[i])

156.3 143.31724392258448
133.9 131.82731891579235
108.1 110.5196328708319
78.9 75.27591039108262
63.2 61.593351300072484
53.6 52.9814886114439
45.7 46.15772233028045
38.5 39.174516872533815
29.4 29.026649658852026
22.1 21.611853934197924
16.1 15.40830508890182
8.03 7.872433275180026
2.85 2.8791093599090924
0.773 0.8921474141681306
0.338 0.3571566041760261
0.169 0.16679284458681545
0.05 0.05220671087255304
2.8e-07 0.0


In [22]:
for i in range(dimM):
    print(l1[i]/s.L, l[i]/s.L)

0.0 0.0
0.079 0.08554061741725977
0.264 0.2818852882521794
0.555 0.5533160768288975
0.7179999999999999 0.7144652784106362
0.809 0.8111775890999076
0.874 0.8798626582908967
0.9209999999999999 0.9296027496870293
0.964 0.9685130825372467
0.986 0.9879229457626771
0.996 0.9969327305056751
1.0 0.9997732660968942
1.0 0.9999923602517709
1.0 0.9999997100568466
1.0 0.9999999835352114
1.0 0.9999999999990291
1.0 1.0
1.0 1.0


In [23]:
for i in range(dimM):
    print(k1[i], k[i])

1.1 1.0086306353163017
1.2 1.181424814181715
1.3 1.3290982629286399
1.4 1.335694225878389
1.6 1.5593253496028225
1.7 1.6803830337646455
1.8 1.818028449080095
1.9 1.9332878475908009
2.1 2.073332117660098
2.4 2.3469886610704633
2.8 2.6797052254833007
3.5 3.4313220971927123
4.7 4.748004917910583
8.0 9.23309097375497
12.2 12.891451408974664
16.8 16.580590463854012
10.0 2.3256087566360413
0.3 0.0


In [24]:
e = np.zeros(dimM) # энерговыделения
a = np.zeros(dimM) # градиент давления
da = np.zeros(dimM) # несоответсвие градиента давления
b = np.zeros(dimM) # градиент температуры
db = np.zeros(dimM) # несоответствие градиента температуры 

Условие гидростатики

In [25]:
for i in range(1, dimM-1):
    mu = ph.MU(x[i], 1 - x[i] -Z, Z)
    a[i] = -ph.G*m[i]*d[i]/pow(r[i], 2)
    da[i] = (p[i+1] - p[i])/2/(r[i+1] - r[i]) + (p[i] - p[i-1])/2/(r[i] - r[i-1]) - a[i]

In [26]:
da[1:dimM-1]/a[1:dimM-1]

array([-0.41677213, -0.06567032,  0.09379674, -0.08518798, -0.10739398,
       -0.05800794,  0.01145504,  0.06947705,  0.03153308, -0.01755841,
        0.05897138,  0.24139306,  0.42967834,  0.36792441,  0.17615776,
        0.32243698])

Условие переноса энергии

In [27]:
for i in range(1, dimM-1):
    rad = -3/16*k[i]*d[i]*l[i]/ph.sigma/pow(t[i], 3)/4/pi/pow(r[i], 2)
    mu = ph.MU(x[i], 1 - x[i] -Z, Z)
    conv = -0.4*mu/ph.kB*ph.m_prot*ph.G*m[i]/pow(r[i], 2)
    if abs(rad) > abs(conv):
        b[i] = rad
    else:
        b[i] = conv
    db[i] = (t[i+1] - t[i])/2/(r[i+1] - r[i]) + (t[i] - t[i-1])/2/(r[i] - r[i-1]) - b[i]

In [28]:

db[1:dimM-1]/b[1:dimM-1]

array([-0.28778159, -0.20928807, -0.22928413, -0.27749335, -0.31891329,
       -0.28764347, -0.32372861, -0.37490914, -0.40796832, -0.44913059,
       -0.48343867, -0.50316461, -0.46892108, -0.3990538 , -0.20716028,
       -0.01067833])

In [29]:
sa = 0
sb = 0
for i in range(1, dimM-1):
    sa += pow(da[i]/a[i], 2)
    sb += pow(db[i]/b[i], 2)
sa, sb    

(0.7319629596483445, 1.9631993737080915)